In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.51 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/73ea66f39d854389ad9bddb8c92b0855



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect
from Model_Unet_github import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_no_intersect_1conv_64_channels_30epoch_unet_github', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-gpu", default='2', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=30, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsect(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 30
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/segnet_no_intersect_1conv_64_channels_30epoch_unet_github
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images_no_intersect already exists, skipping download
[0.05999042 0.94000958]
Instructions for updating:
Colocations handled automatically by placer.
GGG
[None, 32, 128, 1]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


0 conv1: (?, 32, 128, 32)
0 conv2: (?, 32, 128, 32)
1 conv1: (?, 16, 64, 64)
1 conv2: (?, 16, 64, 64)
2 conv1: (?, 8, 32, 128)
2 conv2: (?, 8, 32, 128)
1 h_deconv: (?, ?, ?, 64)
1 h_deconv_concat: (?, ?, ?, ?)
1 h_conv1_post_deconv: (?, ?, ?, 64)
1 h_conv2_post_deconv: (?, ?, ?, 64)
0 h_deconv: (?, ?, ?, 32)
0 h_deconv_concat: (?, ?, ?, ?)
0 h_conv1_post_deconv: (?, ?, ?, 32)
0 h_conv2_post_deconv: (?, ?, ?, 32)
0 outmap: (?, ?, ?, 2)
(?, ?, ?, 2)
loss: ()
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


...

TRAIN: Batch: 0.574622781643343 Loss: 0.0058835885
accuracy = 0.974144
mean IU  = 0.835053
    class # 0 capture rate = 0.973161 
    class # 1 capture rate = 0.989469 
TRAIN: Batch: 0.5785317801579235 Loss: 0.006678369
accuracy = 0.967893
mean IU  = 0.791658
    class # 0 capture rate = 0.966838 
    class # 1 capture rate = 0.986965 
TRAIN: Batch: 0.5824407786725041 Loss: 0.0056543956
accuracy = 0.976779
mean IU  = 0.836445
    class # 0 capture rate = 0.976146 
    class # 1 capture rate = 0.987832 
TRAIN: Batch: 0.5863497771870847 Loss: 0.007896851
accuracy = 0.968114
mean IU  = 0.796478
    class # 0 capture rate = 0.967909 
    class # 1 capture rate = 0.971631 
TRAIN: Batch: 0.5902587757016652 Loss: 0.008652314
accuracy = 0.962299
mean IU  = 0.802211
    class # 0 capture rate = 0.960309 
    class # 1 capture rate = 0.989065 
TRAIN: Batch: 0.5941677742162458 Loss: 0.0067729503
accuracy = 0.976853
mean IU  = 0.841040
    class # 0 capture rate = 0.976042 
    class # 1 capture r

TRAIN: Batch: 0.7661637088577906 Loss: 0.010233125
accuracy = 0.950836
mean IU  = 0.772775
    class # 0 capture rate = 0.947821 
    class # 1 capture rate = 0.988475 
TRAIN: Batch: 0.7700727073723712 Loss: 0.0064838254
accuracy = 0.975414
mean IU  = 0.813788
    class # 0 capture rate = 0.975343 
    class # 1 capture rate = 0.976840 
TRAIN: Batch: 0.7739817058869518 Loss: 0.007799305
accuracy = 0.970553
mean IU  = 0.825721
    class # 0 capture rate = 0.969646 
    class # 1 capture rate = 0.983712 
TRAIN: Batch: 0.7778907044015323 Loss: 0.008525139
accuracy = 0.967770
mean IU  = 0.775766
    class # 0 capture rate = 0.967472 
    class # 1 capture rate = 0.973863 
TRAIN: Batch: 0.7817997029161129 Loss: 0.0058236667
accuracy = 0.976413
mean IU  = 0.825893
    class # 0 capture rate = 0.976116 
    class # 1 capture rate = 0.982021 
TRAIN: Batch: 0.7857087014306935 Loss: 0.007355654
accuracy = 0.965029
mean IU  = 0.796772
    class # 0 capture rate = 0.963408 
    class # 1 capture r

TRAIN: Batch: 0.9577046360722383 Loss: 0.0065788813
accuracy = 0.974116
mean IU  = 0.850024
    class # 0 capture rate = 0.972701 
    class # 1 capture rate = 0.992995 
TRAIN: Batch: 0.9616136345868188 Loss: 0.009286968
accuracy = 0.962590
mean IU  = 0.793169
    class # 0 capture rate = 0.961391 
    class # 1 capture rate = 0.980145 
TRAIN: Batch: 0.9655226331013994 Loss: 0.010597637
accuracy = 0.954406
mean IU  = 0.776933
    class # 0 capture rate = 0.952390 
    class # 1 capture rate = 0.980962 
TRAIN: Batch: 0.96943163161598 Loss: 0.005900061
accuracy = 0.978635
mean IU  = 0.852041
    class # 0 capture rate = 0.978285 
    class # 1 capture rate = 0.984349 
TRAIN: Batch: 0.9733406301305606 Loss: 0.006508127
accuracy = 0.972045
mean IU  = 0.835872
    class # 0 capture rate = 0.971016 
    class # 1 capture rate = 0.986447 
TRAIN: Batch: 0.9772496286451411 Loss: 0.008871911
accuracy = 0.958786
mean IU  = 0.770244
    class # 0 capture rate = 0.957443 
    class # 1 capture rate

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 96.621895%. Class 0 capture: 96.505339%. Class 1 capture: 98.451269%
Character error rate improved, save model
Done with training at epoch 30 sigoptObservation=0.9662189489163772
